- title: Pset 1 answers
- tags: psets
- date: 2018-02-04

In [1]:
# just putting all the imports I'll need in one big go.

import requests, re, random
from datetime import datetime, date, timedelta

## Problem 1

In [2]:
# I'm using this as an easy way of encoding the parameters of the game, 
# so my code can look up a choice and see what beats it.
beats = {"rock": "paper", "paper": "scissors", "scissors": "rock"}
options = list(beats.keys())

def rps():
    computer_choice = random.choice(options)
    player_choice = input("Pick rock, paper or scissors!").lower().strip()
    while player_choice not in options:
        player_choice = input("Please enter rock, paper, or scissors.").lower().strip()
    if player_choice == beats[computer_choice]:
        winner = "You"
    elif computer_choice == beats[player_choice]:
        winner = "I"
    else:
        winner = "Nobody"
    print("I picked {}. {} won!".format(computer_choice, winner))
        
    

In [3]:
rps()

Pick rock, paper or scissors!        rOcK
I picked paper. I won!


In [4]:
rps()

Pick rock, paper or scissors!rock
I picked rock. Nobody won!


In [5]:
rps()

Pick rock, paper or scissors!cat
Please enter rock, paper, or scissors.bear
Please enter rock, paper, or scissors.kraken
Please enter rock, paper, or scissors.rock
I picked paper. I won!


## Problem 2

In [6]:
def check_fixed_holiday(date):
    holidays = ["12-25", "07-04", "01-01", "11-11"]
    datestring = date.strftime("%m-%d")
    return datestring in holidays

def check_weekend(date):
    return date.weekday() >= 5
    
def read_date(datestring = "today"):
    if datestring == "today":
        return date.today()
    return datetime.strptime(datestring, "%m-%d-%Y").date()

def x_days_after(num_days, start_date = "today"):
    start = read_date(start_date)
    return start + timedelta(num_days)

def calculate_days(num_days, start_date = "today"):
    try:
        candidate = x_days_after(num_days, start_date)
    except:
        return None
    while check_weekend(candidate) or check_fixed_holiday(candidate):
        candidate += timedelta(1)
    return candidate.strftime("%a, %B %d, %Y")

In [7]:
print(calculate_days("cats are the best", 5))

None


In [8]:
calculate_days(3, "01-16-2019")

'Mon, January 21, 2019'

In [9]:
calculate_days(19, "06-15-2019")

'Fri, July 05, 2019'

## Problem 3

In [10]:
endpoint = "https://api.case.law/v1/cases/"

pattern = r"\d+ U\.S\. \d+"

def get_opinion_texts(api_response):
    try:
        ops = api_response["results"][0]["casebody"]["data"]["opinions"]
    except:
        return None
    return [x["text"] for x in ops]

import json
def ppjson(jsonfile):
    return json.dumps(jsonfile, sort_keys=True, indent=4)
    

def cite_finder(cite):
    resp = requests.get(endpoint, params={"cite": cite, "full_case": "true"}).json()
    # we used this line in class to look at the JSON for an example of how to inspect a json
    # and see what its structure is. 
    # print(ppjson(resp))
    opinions = get_opinion_texts(resp)
    if opinions:
        allcites = []
        for opinion in opinions:
            opcites = re.findall(pattern, opinion)
            allcites.extend(opcites)
        filtered = list(set(allcites))
        filtered.sort()
        return filtered
    return None

In [11]:
cite_finder("231 Ill.2d 474")

['387 U.S. 136',
 '419 U.S. 102',
 '424 U.S. 1',
 '429 U.S. 252',
 '508 U.S. 520',
 '509 U.S. 43']

In [12]:
print(cite_finder("231 Ill.2d 475"))

None


In [13]:
cite_finder("215 Ill.2d 219")

['339 U.S. 594', '387 U.S. 136', '467 U.S. 837', '538 U.S. 803']